In [1]:
%gui qt5

In [2]:
%config Completer.use_jedi = False

In [16]:
%load_ext autoreload
%autoreload 2

In [3]:
import napari

In [4]:
from dask.distributed import Client

In [5]:
from dask import dataframe as dd

In [6]:
import pandas as pd
from pathlib import Path

In [40]:
from itertools import groupby

In [7]:
import numpy as np

In [8]:
from sklearn.neighbors import NearestNeighbors

In [9]:
from pynndescent import NNDescent

In [10]:
from pysmFISH.stitching import organize_square_tiles
from pysmFISH.logger_utils import selected_logger
from pysmFISH.io import open_consolidated_metadata
from pysmFISH.configuration_files import load_experiment_config_file

In [21]:
from pysmFISH.configuration_files import load_analysis_config_file

In [17]:
from pysmFISH.stitching import stitch_using_microscope_fov_coords
from pysmFISH.stitching import remove_overlapping_dots_fov
from pysmFISH.stitching import clean_from_duplicated_dots

In [123]:
from pysmFISH.io import simple_output_plotting

In [12]:
client = Client()
client

/Users/simone.codeluppi/opt/miniconda3/envs/test_d/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58893 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:58894 Dashboard: http://127.0.0.1:58893/status,Cluster Workers: 4 Cores: 16 Memory: 68.72 GB


In [82]:
experiment_fpath = Path('/Users/simone.codeluppi/Documents/data_analysis_jlabs_sc/eel_human_embryo_project/LBEXP20210304_EEL_HE_1500um/data/LBEXP20210304_EEL_HE_1500um//')
parsed_raw_data_fpath = experiment_fpath / 'LBEXP20210304_EEL_HE_1500um_img_data.zarr'
consolidated_grp = open_consolidated_metadata(parsed_raw_data_fpath)
experiment_info = load_experiment_config_file(experiment_fpath)
analysis_parameters = load_analysis_config_file(experiment_fpath)

In [85]:
round_num =1
tiles_org = organize_square_tiles(experiment_fpath,experiment_info,consolidated_grp,round_num)
tiles_org.run_tiles_organization()

In [86]:
tiles_org.overlapping_regions.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

# Eliminate overlapping dots by cpls

In [87]:
# PROCESSING PARAMETERS
registration_channel = experiment_info['StitchingChannel']
key = Path(experiment_fpath).stem + '_Hybridization01_' + registration_channel + '_fov_0'
fovs = consolidated_grp[key].attrs['fields_of_view']
img_width = consolidated_grp[key].attrs['img_width']
img_height = consolidated_grp[key].attrs['img_height']
registration_reference_hybridization = analysis_parameters['RegistrationReferenceHybridization']
selected_genes = 'below3Hdistance_genes'
correct_hamming_distance = 'zeroHdistance_genes' 

In [160]:
# Prepare the dataframe
select_genes = 'below3Hdistance_genes'
stitching_selected = 'microscope_stitched'
same_dot_radius = 100
r_tag = 'r_px_' + stitching_selected
c_tag = 'c_px_' + stitching_selected

In [161]:
unfolded_overlapping_regions_dict = {key:value for (k,v) in tiles_org.overlapping_regions.items() for (key,value) in v.items()}
corrected_overlapping_regions_dict = {}
for key, value in unfolded_overlapping_regions_dict.items():
    corrected_overlapping_regions_dict[key] = np.array(value)-img_width

In [162]:
%%time
all_futures = []

for cpl,chunk_coords in corrected_overlapping_regions_dict.items():
    future = client.submit(remove_overlapping_dots_fov,
                            cpl = cpl,
                            chunk_coords=chunk_coords,
                            experiment_fpath=experiment_fpath,
                            stitching_selected=stitching_selected,
                            select_genes=select_genes,
                            same_dot_radius = same_dot_radius)

    all_futures.append(future)

to_remove = client.gather(all_futures) 

CPU times: user 28.5 s, sys: 4.09 s, total: 32.6 s
Wall time: 5min 24s


In [163]:
to_remove_list = [el for tg in to_remove for el in tg]

In [164]:
len(to_remove_list)

793539

In [165]:
removed_dot_dict = {k: list(v) for k, v in groupby(sorted(to_remove_list), key=lambda x: int(x.split('_')[0]))}

In [146]:
for fov in fovs:
    if fov not in removed_dot_dict.keys():
        removed_dot_dict.update({fov:[]})


for fov,dots_id_to_remove in removed_dot_dict.items():
    future = client.submit(clean_from_duplicated_dots,
                            fov = fov,
                            dots_id_to_remove=dots_id_to_remove,
                            experiment_fpath=experiment_fpath)

    all_futures.append(future)

_ = client.gather(all_futures)

In [147]:
simple_output_plotting(experiment_fpath, stitching_selected, select_genes, client)

# Check removed in fov

In [166]:
cpl= (0,11)
counts1_fpath = list((experiment_fpath / 'tmp' / 'registered_counts').glob('*decoded*_fov_' + str(cpl[0]) + '.parquet'))[0]
counts2_fpath = list((experiment_fpath / 'tmp' / 'registered_counts').glob('*decoded*_fov_' + str(cpl[1]) + '.parquet'))[0]

In [167]:
counts1_df = pd.read_parquet(counts1_fpath)
counts2_df = pd.read_parquet(counts2_fpath)

In [168]:
removed_dots_df = counts2_df.loc[counts2_df.barcode_reference_dot_id.isin(removed_dot_dict[cpl[1]]),:]

In [169]:
c2_cleaned_df = counts2_df.loc[~counts2_df.barcode_reference_dot_id.isin(removed_dot_dict[cpl[1]]),:]

In [170]:
c1 = counts1_df.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()
c2 = counts2_df.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()
r2 = removed_dots_df.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()
c2_cleaned = c2_cleaned_df.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()

In [171]:
vw = napari.Viewer()
_ = vw.add_points(c1, name='c1', size=5,symbol='o',face_color='cyan')
_ = vw.add_points(c2, name='c2', size=5,symbol='+',face_color='magenta')
_ = vw.add_points(r2, name='c2', size=5,symbol='+',face_color='yellow')
_ = vw.add_points(c2_cleaned, name='c2_cleaned', size=5,symbol='+',face_color='green')

# Run only one cpl

In [174]:
cpl= (0,11)
chunk_coords = corrected_overlapping_regions_dict[cpl]
same_dot_radius = 5
removed_dots = remove_overlapping_dots_fov(
                            cpl = cpl,
                            chunk_coords=chunk_coords,
                            experiment_fpath=experiment_fpath,
                            stitching_selected=stitching_selected,
                            select_genes=select_genes,
                            same_dot_radius = same_dot_radius)

In [176]:
counts1_fpath = list((experiment_fpath / 'tmp' / 'registered_counts').glob('*decoded*_fov_' + str(cpl[0]) + '.parquet'))[0]
counts2_fpath = list((experiment_fpath / 'tmp' / 'registered_counts').glob('*decoded*_fov_' + str(cpl[1]) + '.parquet'))[0]
counts1_df = pd.read_parquet(counts1_fpath)
counts2_df = pd.read_parquet(counts2_fpath)

In [177]:
removed_dots_df = counts2_df.loc[counts2_df.barcode_reference_dot_id.isin(removed_dot_dict),:]

In [178]:
c2_cleaned_df = counts2_df.loc[~counts2_df.barcode_reference_dot_id.isin(removed_dot_dict[cpl[1]]),:]

In [179]:
c1 = counts1_df.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()
c2 = counts2_df.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()
r2 = removed_dots_df.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()
c2_cleaned = c2_cleaned_df.loc[:,['r_px_microscope_stitched','c_px_microscope_stitched']].to_numpy()

In [180]:
vw = napari.Viewer()
_ = vw.add_points(c1, name='c1', size=5,symbol='o',face_color='cyan')
_ = vw.add_points(c2, name='c2', size=5,symbol='+',face_color='magenta')
_ = vw.add_points(r2, name='c2', size=5,symbol='+',face_color='yellow')
_ = vw.add_points(c2_cleaned, name='c2_cleaned', size=5,symbol='+',face_color='green')